In [1]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0

def create_counter(env):
    global no_of_counters
    no_of_counters += 1    #increments the number of counters
    arrive = env.now         #new arrive time as the counter just opened
    new_counter = simpy.Resource(env, capacity = 1)  #creates a new counter
    print( '%7.4f Counter%02d: Opened' % (env.now, no_of_counters) )
    return new_counter

def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          counter = create_counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def customer(env, count_cus, name, counter, time_in_bank):
    arrive = env.now   
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        print ( "patience of %s: %7.4f" % (name, patience) )
        yield req | env.timeout(patience)
        wait = env.now - arrive

        if ( env.now == (arrive + patience)  and  no_of_counters <= 20 ):
              env.process.interrupt()
              patience = 0

        elif ( env.now == (arrive + patience)  and  no_of_counters > 20 ):
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            yield env.timeout(patience)
            count_cus -= 1
            patience = 0

        try:
          # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
            count_cus -= 1

        except simpy.Interrupt:
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            #creating a new counter
            counter = create_counter(env)  #this creates a new counter
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
            count_cus -= 1


        # if (arrive + patience != env.now):
        #     # We got to the counter
        #     print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        #     tib = random.expovariate(1.0 / time_in_bank)
        #     yield env.timeout(tib)
        #     print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
        #     count_cus -= 1
            

        # elif ( arrive + patience == env.now  and  no_of_counters <= 20 ):  #checks if counter no. is<20 and previous counter occupied
        #     print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
        #     #creating a new counter
        #     counter = create_counter(env)  #this creates a new counter
        #     # env.process( customer(env, count_cus, name, counter, time_in_bank=12.0) )
        #     # wait = 0         #waiting time 0 as the counter just opened
        #     # print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        #     tib = random.expovariate(1.0 / time_in_bank)
        #     yield env.timeout(tib)
        #     print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
        #     count_cus -= 1

        # else:
        #     # We reneged
        #     print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
        #     yield env.timeout(patience)
        #     count_cus -= 1

    global count_customer
    count_customer = count_cus
    yield env.timeout(patience)

print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = create_counter(env)
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
patience of Customer00:  1.0500
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
patience of Customer01:  2.4729
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
patience of Customer02:  1.1739


TypeError: ignored

In [3]:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0
give_up = False


def create_counter(env):
    global no_of_counters
    no_of_counters += 1    #increments the number of counters
    arrive = env.now         #new arrive time as the counter just opened
    new_counter = simpy.Resource(env, capacity = 1)  #creates a new counter
    print( '%7.4f Counter%02d: Opened' % (env.now, no_of_counters) )
    return new_counter


def source(env, number, interval, counter):
    global count_customer
    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          counter = create_counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):  
    global give_up

    if not give_up:
        arrive = env.now      
        print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.event()
        # print ( "patience of %s: %7.4f" % (name, patience) )
        wait = env.now - arrive

        if req in results:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
            give_up = False
            count_cus -= 1
            
        elif ( req not in results  and  no_of_counters <= 20 ):  #checks if counter no. is<20 and previous counter occupied
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            give_up = True
            counter = create_counter(env)  #this creates a new counter
            env.process( customer(env, count_cus, name, counter, time_in_bank=12.0) )
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, no_of_counters) )
            count_cus -= 1

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            yield env.timeout(patience)
            count_cus -= 1

    global count_customer
    count_customer = count_cus


print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = create_counter(env)
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
23.7507 Customer01: Served by Counter 01
23.7507 Customer02: Waited 11.024
30.3272 Customer02: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
35.5708 Customer05: Here I am
43.4463 Customer03: Served by Counter 01
43.4463 Customer04: Waited  8.144
46.0657 Customer06: Here I am
46.4346 Customer04: Served by Counter 01
46.4346 Customer05: Waited 10.864
46.5129 Customer05: Served by Counter 01
46.5129 Customer06: Waited  0.447
54.9638 Customer07: Here I am
66.1805 Customer06: Served by Counter 01
66.1805 Customer07: Waited 11.217
66.9417 Customer08: Here I am
68.2083 Customer07: Served by Counter 01
68.2083 Customer08: Waited  1.267
69.3763 Customer08: Served by Counter 01
98.4569 Customer0

In [11]:
#normal resource - def process_req (self, prio): - if req in results:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers
customer_in_counter = 0

class Counter(object):
    instances = []
    def __init__(self, env):
        self.env = env
        self.create_counter()
        Counter.instances.append(self)
        global no_of_counters
        no_of_counters += 1

    def create_counter(self):
        self.new_counter = simpy.Resource( self.env, capacity = 1 )  #creates a new counter
        print( '%7.4f Counter%02d: Opened' % (env.now, no_of_counters) )

    # def process_req (self, prio):

    #   with self.new_counter.request( priority = prio ) as req:
    #       patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
    #       # Wait for the counter or abort at the end of our tether
    #       results = yield req | env.timeout(patience)
    #       if req in results:
    #         counter_num = prio
    #       else:
    #         counter_num = prio - 1
    #   return counter_num, results



def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          counter = Counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    # counter_num, results = counter.process_req( count_cus )

    with counter.new_counter.request( priority = count_cus ) as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, count_cus) )
            count_cus -= 1
            

        elif ( req not in results  and  no_of_counters <= 20 ):  #checks if counter no. is<20 and previous counter occupied
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            counter = Counter(env) 
            counter_num, results = counter.process_req( count_cus )
            # env.process( customer(env, count_cus, name, counter, time_in_bank=12.0) )
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, count_cus - 1) )
            count_cus -= 1

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            # yield env.timeout(patience)
            count_cus -= 1

    global count_customer
    count_customer = count_cus


# Setup and start the simulation
print('Bank renege')

random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)

# Start processes and run
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter00: Opened
 0.0000 Customer00: Here I am


TypeError: ignored